# For the 2nd paper
## deuterium acceleration energy is 100 kV


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## ENERGY

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import os
from matplotlib.ticker import AutoMinorLocator

os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']


def fun_getNeutronEnergyVSEmissionAngle(energy, angle):
    # returns xRad,y,angle,intp_Y_av : xRad is the angle in radian for the formula, y is the neutron energy at xRad (so this is the "real" value)
    #       angle is the angle in emission radian, intp_Y_av is the neutron energy in MeV for MCNP
    # energy is the energy of the incoming deuterium ion
    # angle is the emission angle in radian
    def mathfun_Eout_vs_angle(energy,angle):
        # energy is the impinging deuterons energy
        # angle is the emission angle
        res = 0.3542 * np.sqrt(energy) * np.cos(angle) + ( np.sqrt( 2.031 * energy * (np.cos(angle))**2 + 39.6502 + 4.0291 * energy ) ) / ( 4.02359755 )
        return res**2

    a = np.arange(0.0,181,0.1)

    #energy = 0.05 # D beam energy in MeV
    xRad = []
    y = []
    for angl in a:
        angl = angl / 180.0 * np.pi
        xRad.append(angl)
        y.append(mathfun_Eout_vs_angle(energy,angl))


    # lookup the xs data for each emission angle --> interpolate between the two closest values
    def find_nearest(array,value):
        idx = (np.abs(array-value)).argmin()
        return idx,array[idx]

    # find where in the x data (emission angle) the angle that is desired for the MCNP file are located
    xRad = np.asarray(xRad)
    idxAngle = []
    for val in angle:
        idx, nearest = find_nearest(xRad, val)
        idxAngle.append(idx) # append nearest value where that angle is located in the datafile

    # interpolate between the angles: take the interval between two values
    idxAngleCenter = []
    for ii in range(0,len(idxAngle)-1):
        v2 = idxAngle[ii + 1]
        v1 = idxAngle[ii]
        v3 = round((v1+v2)/2,0)
        idxAngleCenter.append(v3)

    # interpolate between the angles: intervals where to evaluate the angles
    idxAngleRange = []
    for ii in range(0,len(idxAngle)):
        if ii is 0:
            idxAngleRange.append([0,idxAngleCenter[ii]])
        elif ii < len(idxAngle)-1:
            idxAngleRange.append([idxAngleCenter[ii-1],idxAngleCenter[ii]])
        else:
            idxAngleRange.append([idxAngleCenter[ii-1], idxAngle[-1]])

    # interpolate between the angles! for example 0 - 10 degree
    intp_Y_av = [] # [-1.0] * len(angle)# interpolated yield values as the average between the two angles (absolute, but not normalized!)
    for pair in idxAngleRange:
        #ii = np.where(idxAngleRange == pair)[0][0]
        val1 = int(pair[0])
        val2 = int(pair[1])
        res = np.mean(y[val1:val2])
        intp_Y_av.append(res)

    return xRad,y,angle,intp_Y_av

angle_start = 0.0
angle_stop = 180.0
angle_step = 10.0
# energies to plot
energies = np.asarray([100]) / 1000# MeV


angle_rad = list(map(np.cos,np.arange(angle_start,angle_stop + 1.0, angle_step)))
angle = np.arange(angle_start, angle_stop + 1.0, angle_step)
# convert to radian
angle = (angle / 360.0) * 2 * np.pi
mu = np.cos(angle) # this is the discretization chosen for the MCNP source probability

data = pd.DataFrame()
# columns: energy in MeV angle in degree
for energy in energies:
    xRad,y,angle,intp_Y_av = fun_getNeutronEnergyVSEmissionAngle(energy, angle)
    col = str(energy)
    data[col] = y
    data['angle_'+col] = (xRad/np.pi) * 180

# make sure the angles are the same
cols_angle = [c for c in data.columns if 'angle' in c]
for c in cols_angle:
    assert len(data[c]) == len(xRad)
data = data.drop(columns=cols_angle)
data['angle'] = (xRad/np.pi) * 180
# poly = np.polyfit(data['angle'].values,data['energy'].values,10)

# X = np.linspace(0, 180, 100)
# poly_y = np.poly1d(poly)(X)
# # print(X)

# PLOT
fig, ax1 = plt.subplots(figsize=(8, 6))

fs = 20
markersize = 8
colors = ['darkgreen', 'darkorange', 'darkblue', 'darkred']

for energy, color in zip(energies, colors):
    label = int(energy*1000)
    ax1.plot(data['angle'].values,data[str(energy)].values,'-', label=label, color=color)
    
ax1.set_xlabel(r"\textbf{Emission angle [degree]}", fontsize=fs)
ax1.set_ylabel(r"\textbf{Neutron energy [MeV]}", fontsize=fs)
ax1.tick_params('x', colors='black', labelsize=fs-2)
ax1.tick_params(axis='y', labelsize=fs-2)
ax1.set_ylim(2.1,3.0)
ax1.set_yticks(np.arange(2.0,3.1,0.2))
# ax1.set_xlim()
ax1.set_xticks(np.arange(0,180+30,30))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)
leg1 = plt.legend(loc='upper right', fontsize=fs-4)
leg1.set_title(r"\textbf{Deuterium energy [keV]}", prop = {'size': fs-4})
ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')

plt.grid(True)
plt.show()

data_energy = data

## YIELD


In [ ]:
xsDatafile = '/Users/hkromer/02_PhD/02_Data/07.neutron_output/relative_yield_DD.xlsx'

data_rel = pd.read_excel(xsDatafile, skiprows=1)
data_rel = data_rel.iloc[:,1:]
cols = data_rel.columns.tolist()

cols = ['angle_radians' if 'Unnamed' in str(c) else str(c*1000) for c in cols ] # columns are in keV

data_rel.columns = cols
cols_to_keep = ['angle_radians', '60.0', '100.0', '150.0']
data_rel = data_rel[cols_to_keep]
data_rel['angle_degree'] = (data_rel['angle_radians']/np.pi) * 180

# columns with data
E_cols = [c for c in data_rel.columns if '0' in c]




# get intermediate values
energies = np.arange(60.0,160.0,10.0)
energies = [str(e) for e in energies]
for e in energies:
    if e in data_rel.columns:
        pass
    else:
        data_rel[e] = np.nan
E_cols_new = [c for c in data_rel.columns if '0' in c]
data_rel_new = data_rel[E_cols_new].copy().T
data_rel_new.reset_index(inplace=True)
data_rel_new['index'] = data_rel_new['index'].astype(float)
data_rel_new = data_rel_new.sort_values(by='index')
data_rel_new.reset_index(inplace=True, drop=True)
data_rel_new.set_index('index', inplace=True)
data_rel_new = data_rel_new.interpolate(method='piecewise_polynomial')

data_rel_new = data_rel_new.T
data_rel_new['angle_degree'] = data_rel['angle_degree']
# print(data_rel_new)

data_rel_new.columns = [str(c) for c in data_rel_new.columns]
E_cols_new = [c for c in data_rel_new.columns if '0' in c]



# interpolate with polynomial fit
data_rel_interp = pd.DataFrame()

for c in E_cols_new:
    poly = np.polyfit(data_rel_new['angle_degree'].values, data_rel_new[c].values,10)
    X = np.arange(0, 180+0.1, 0.1)
    poly_y = np.poly1d(poly)(X)
    data_rel_interp['angle_degree'] = X
    data_rel_interp[c] = poly_y
    # normalize to 90 degree
    normval = data_rel_interp[ data_rel_interp['angle_degree'] == 90 ].loc[:,c].values
    data_rel_interp[c] = data_rel_interp[c]/normval
    
# plot
for c in E_cols_new:
    plt.plot(data_rel_interp['angle_degree'].values, data_rel_interp[c].values, label=c)
    
plt.legend()
plt.show()
# data_rel_interp.to_csv('/Users/hkromer/02_PhD/02_Data/07.neutron_output/relative_yield_DD_interpolated_normed.csv')
data_angular = data_rel_interp

### Combined plot

In [ ]:
# prepare data

# column names in the dataframe to plot
energy_e = '0.1' # deuterium energy in MeV
energy_a = '100.0'
print(data_energy.head())
data_angular = data_angular[['angle_degree', energy_a]]
print(data_angular.head())

In [ ]:
# PLOT
fig, ax1 = plt.subplots(figsize=(8, 6))

fs = 20
markersize = 8

# Energy on left y axis
p1=ax1.plot(data_energy['angle'].values, data_energy[energy_e].values, color='darkblue', markersize=markersize, label=r'Neutron energy', linewidth=2.5)
ax1.set_xlabel(r"\textbf{Emission angle [°]}", fontsize=fs)
ax1.set_ylabel(r"\textbf{Neutron energy [MeV]}", fontsize=fs, color='darkblue')
ax1.tick_params('x', colors='black', labelsize=fs-2)
ax1.tick_params(axis='y', labelcolor='darkblue', labelsize=fs-2)
ax1.set_ylim(2.1,2.9)
ax1.set_yticks(np.arange(2.1,3.1,0.2))
# ax1.set_xlim()
ax1.set_xticks(np.arange(0,180+30,30))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)



ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis


# Yield on right y axis
# poly = np.polyfit(data_angular['angle_degree'].values,data_angular[energy_a].values,10)
# X = np.linspace(0, 180, 100)
# poly_y = np.poly1d(poly)(X)
p2=ax2.plot(data_angular['angle_degree'].values,data_angular[energy_a].values, '--', color='darkred', markersize=markersize, label=r'Relative neutron yield', linewidth=2.5)
ax2.set_ylabel(r"\textbf{Relative yield [-]}", fontsize=fs, color='darkred')
ax2.tick_params(axis='y', labelcolor='darkred', labelsize=fs-2)
ax2.set_ylim(0.9,2.4)
ax2.set_yticks(np.arange(0.8,2.6,0.4))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax2.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax2.yaxis.set_minor_locator(minor_locator)


ps = p1+p2
labs = [p.get_label() for p in ps]
ax2.legend(ps, labs, loc='best', fontsize=fs-4)
ax1.grid(which="minor",ls="--", color='0.65')
ax1.grid(which="major",ls="-", color='0.55')
# ax1.grid(None)
plt.tight_layout()

# plt.savefig(f'./{figname}.pdf')
# plt.savefig(f'./{figname}_600.png',dpi=600)
# plt.savefig(f'./{figname}_1200.png',dpi=1200)
plt.show()

# For the 2nd paper
## deuterium acceleration energy is 80 kV
- Neutron energy and yield distribution, data is from Benoit who took it from Csikai

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# import yield file
fname_yield = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/Figures/energy_and_yield_vs_angle/data_from_Benoit/80KeV_yield.csv'

# import energy file
# emission angle in degree, neutron energy in MeV
fname_energy = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/Figures/energy_and_yield_vs_angle/data_from_Benoit/80kV_energy.xlsx'

data_yield = pd.read_csv(fname_yield, header=0, delimiter=';')
data_yield.columns = ['emission_angle', 'counts']
print(data_yield.head())
print(data_yield.info())

data_energy = pd.read_excel(fname_energy, header=None)
data_energy.columns = ['emission_angle', 'neutron_energy']
print(data_energy.head())
print(data_energy.info())

In [ ]:
plt.plot(data_yield['emission_angle'].values, data_yield['counts'].values)
plt.show()

In [ ]:
plt.plot(data_energy['emission_angle'].values, data_energy['neutron_energy'].values)
plt.show()

## Normalize yield wrt 90 degree

In [ ]:
counts_90 = data_yield[ data_yield['emission_angle'] == 90  ]['counts'].values[0]
data_yield['R_norm'] = data_yield['counts'] / counts_90

plt.plot(data_yield['emission_angle'].values, data_yield['R_norm'].values)
plt.show()

# Interpolate values for a nicer plot

In [ ]:



poly = np.polyfit(data_yield['emission_angle'].values,data_yield['R_norm'].values,10)

X = np.linspace(0, 180, 100)
poly_y = np.poly1d(poly)(X)
# print(X)

# X = np.arange(0,180,1)
plt.plot(X, poly_y)
# plt.plot(data_yield['emission_angle'].values, data_yield['R_norm'].values)
plt.show()

# Plot all in one figure

In [ ]:
from scipy.interpolate import splrep, splev
from scipy.interpolate import splrep, splev
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from matplotlib.ticker import AutoMinorLocator

os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
%matplotlib inline

# import yield file
fname_yield = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/Figures/energy_and_yield_vs_angle/data_from_Benoit/80KeV_yield.csv'

# import energy file
# emission angle in degree, neutron energy in MeV
fname_energy = '/Users/hkromer/02_PhD/04_Publiations/01_PaperDrafts/2019/01.CharacterizationPaper/Figures/energy_and_yield_vs_angle/data_from_Benoit/80kV_energy.xlsx'

data_yield = pd.read_csv(fname_yield, header=0, delimiter=';')
data_yield.columns = ['emission_angle', 'counts']
counts_90 = data_yield[ data_yield['emission_angle'] == 90  ]['counts'].values[0]
data_yield['R_norm'] = data_yield['counts'] / counts_90
print(data_yield.head())
print(data_yield.info())

print(' ')
data_energy = pd.read_excel(fname_energy, header=None)
data_energy.columns = ['emission_angle', 'neutron_energy']
print(data_energy.head())
print(data_energy.info())


# PLOT
fig, ax1 = plt.subplots(figsize=(8, 6))

fs = 20
markersize = 8

# Energy on left y axis
p1=ax1.plot(data_energy['emission_angle'].values, data_energy['neutron_energy'].values, color='darkblue', markersize=markersize, label=r'Neutron energy')
ax1.set_xlabel(r"\textbf{Emission angle [degree]}", fontsize=fs)
ax1.set_ylabel(r"\textbf{Neutron energy [MeV]}", fontsize=fs, color='darkblue')
ax1.tick_params('x', colors='black', labelsize=fs-2)
ax1.tick_params(axis='y', labelcolor='darkblue', labelsize=fs-2)
ax1.set_ylim(2.1,2.9)
ax1.set_yticks(np.arange(2.1,3.1,0.2))
# ax1.set_xlim()
ax1.set_xticks(np.arange(0,180+30,30))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)



ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis


# Yield on right y axis
poly = np.polyfit(data_yield['emission_angle'].values,data_yield['R_norm'].values,10)
X = np.linspace(0, 180, 100)
poly_y = np.poly1d(poly)(X)
p2=ax2.plot(X, poly_y, '--', color='darkred', markersize=markersize, label=r'Relative neutron yield')
ax2.set_ylabel(r"\textbf{Relative yield [-]}", fontsize=fs, color='darkred')
ax2.tick_params(axis='y', labelcolor='darkred', labelsize=fs-2)
ax2.set_ylim(0.9,1.9)
ax2.set_yticks(np.arange(0.8,2.2,0.2))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax2.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax2.yaxis.set_minor_locator(minor_locator)


ps = p1+p2
labs = [p.get_label() for p in ps]
ax2.legend(ps, labs, loc='best', fontsize=fs-4)
ax1.grid(which="minor",ls="--", color='0.65')
ax1.grid(which="major",ls="-", color='0.55')
# ax1.grid(None)
plt.tight_layout()

# plt.savefig(f'./{figname}.pdf')
# plt.savefig(f'./{figname}_600.png',dpi=600)
# plt.savefig(f'./{figname}_1200.png',dpi=1200)
plt.show()